<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/my_colab_gpu_topk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# task
1.给定850万条规模的数据文件，每条数据是最大128维度的整型id向量 （称为doc），id取值范围是0-50000，给定一个最大128维的整型id向量（称为query），求query与doc全集内各数据的交集个数topk（k=100）

# reference
1. https://www.youtube.com/watch?v=cOBtkPsgkus
2. https://raw.githubusercontent.com/cteqeu/HAC/main/GPU/cuda_profile.cu
3. https://docs.nvidia.com/cuda/cuda-runtime-api/index.html
4. https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html
5. https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html



In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64/nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt update
!apt install ./nsight-systems-2023.2.3_2023.2.3.1001-1_amd64.deb
!apt --fix-broken install


In [ ]:
!wget "https://bj.bcebos.com/v1/ai-studio-online/9805dd2d2e8e472693efac637628e16b9f9c5be0fe30438bb4a80de3b386781a?responseContentDisposition=attachment%3B%20filename%3DSTI2_1017.zip&authorization=bce-auth-v1%2F5cfe9a5e1454405eb2a975c43eace6ec%2F2023-10-18T12%3A42%3A27Z%2F-1%2F%2F6b5388dcd9013bc9b340bb1806476afa938ce0c65f2f595e1a75f529e90e4187" -O STI2_1017.zip

In [ ]:
!rm -rf STI2 && unzip STI2_1017.zip && mv STI2\ 2 STI2

In [4]:
!sh STI2/build.sh

build success


### just dev/test with colab vim IDE ❤ 🐑 🐑 🐑

In [26]:
!mkdir -p querys

In [27]:
!touch querys/query0.txt

In [44]:
!rm docs.txt && touch docs.txt

In [78]:
!nvcc STI2/src/main.cpp STI2/src/topk.cu -o ./query_doc_scoring  -I STI2/src -L/usr/local/cuda/lib64 -lcudart -lcuda  -O3 -g

In [71]:
!./query_doc_scoring ./docs.txt ./querys ./res.txt

start get topk
query0.txt:10
query_size: 1
doc_size: 21
read file cost 0 ms 
end
query_idx:0 	 query_on_shm[query_idx]:10	 j:0	 doc_segment[j]:10	 tmp_score:1.0000000000 
query_idx:0 	 query_on_shm[query_idx]:10	 j:0	 doc_segment[j]:10	 tmp_score:1.0000000000 
query_idx:0 	 query_on_shm[query_idx]:10	 j:0	 doc_segment[j]:1	 tmp_score:0.0000000000 
query_idx:0 	 query_on_shm[query_idx]:10	 j:1	 doc_segment[j]:10	 tmp_score:2.0000000000 
query_idx:0 	 query_on_shm[query_idx]:10	 j:1	 doc_segment[j]:10	 tmp_score:1.0000000000 
query_idx:0 	 query_on_shm[query_idx]:10	 j:2	 doc_segment[j]:10	 tmp_score:3.0000000000 
query_idx:0 	 query_on_shm[query_idx]:10	 j:2	 doc_segment[j]:10	 tmp_score:2.0000000000 
query_idx:0 	 query_on_shm[query_idx]:10	 j:3	 doc_segment[j]:1	 tmp_score:2.0000000000 
query_idx:0 	 query_on_shm[query_idx]:10	 j:4	 doc_segment[j]:10	 tmp_score:3.0000000000 
doc_id:0	 scores[doc_id]:1.0000000000	 tmp_score:1.0000000000	 query_len:1	 doc_lens[doc_id]:1
doc_id:2	 scores

In [72]:
!cat res.txt

222
0	2	4	1	3	5	6	7	8	9


In [74]:
!cat STI2/src/{main.cpp,topk.cu,topk.h}

/*
 * SPDX-FileCopyrightText: Copyright (c) <year> NVIDIA CORPORATION & AFFILIATES. All rights reserved.
 * SPDX-License-Identifier: LicenseRef-NvidiaProprietary
 *
 * NVIDIA CORPORATION, its affiliates and licensors retain all intellectual
 * property and proprietary rights in and to this material, related
 * documentation and any modifications thereto. Any use, reproduction,
 * disclosure or distribution of this material and related documentation
 * without an express license agreement from NVIDIA CORPORATION or
 * its affiliates is strictly prohibited.
 */

#include <cuda.h>
#include <chrono>
#include <vector>
#include <random>
#include <cassert>
#include <fstream>
#include <sstream>
#include <iostream>
#include <algorithm>
#include <sys/time.h>
#include <stdio.h>
#include <dirent.h>
#include "topk.h"
#include <sys/stat.h>

std::vector<std::string> getFilesInDirectory(const std::string& directory)
{
    std::vector<std::string> files;
    DIR* dirp = opendir(directory.c_str());
    

In [77]:
!cat ./querys/query0.txt  ./docs.txt ./res.txt

10
10
1812,3,4,10,1,4
10,10,10
42,12,11,10,3,4
1,10,10,1,10
49,10,4,5
2224
41031
13
1408
40206
21, 22, 23, 25, 26, 27, 29, 1711, 1757, 1841, 1910, 1967, 1971, 1991, 2005, 2051, 2059, 2079, 2117, 2128, 2141, 2188, 2306, 2352, 2453, 2470, 2516, 2518, 2562, 2564, 2573, 2631, 2649, 2650, 2656, 2658, 2682, 2721, 2725, 2730, 2751, 2753, 2756, 2835, 2870, 2875, 2886, 2928, 2954, 3010, 3034, 3035, 3037, 3054, 3110, 3114, 3138, 3156, 3167, 3221, 3248, 3250, 3254, 3273, 3274, 3284, 3294, 3318, 3330, 3342, 3363, 3374, 3427, 3452, 3468, 3472, 3491, 3503, 3530, 3535, 3553, 3588, 3596, 3609, 3667, 3682, 3689, 3719, 3722, 3735, 3752, 3771, 3781, 3793, 3831, 3859, 3883, 3887, 3932, 4021, 4055, 4084, 4098, 4238, 4262, 4295, 4299, 4373, 4499, 4626, 4650, 4791, 4796, 4813, 4848, 5060, 5088, 5174, 5278, 5463, 5776, 22466, 22471, 22482, 30054, 33641, 39222
21, 22, 23, 25, 26, 27, 29, 1660, 1668, 1711, 1834, 1841, 1967, 1971, 1991, 2005, 2051, 2059, 2079, 2117, 2188, 2286, 2306, 2352, 2453, 2516, 2518, 2520